## 1. Przetwarzenie danych wejściowych
Czyta z pliku "letter-recognition.data" dane tych liter, które mają zostać uwzględnione w badaniu. Następnie umieszcza na liście y etykiety klas (litery), a na liście X, wektory cech tychże klas.

In [14]:
import numpy as np
X = [] #lista wektorów reprezentujących litery
y = [] #lista etykiet klas (litery)
litery = ('P','R','O','S','I','A','C','Z','E','K')

with open("letter-recognition.data", "r") as f:
    for line in f:
        if line[0] in litery:
            
            #odrzua znaki końca linii
            line = line.rstrip('\n')
            
            #przecinek jest znakiem rozdzielającym wyrazy (wartości)
            line = line.split(",")
            
            #pierwszy znak w linii to etykieta klasy - dodaje do y
            y.append(line[0])
            
            #pozostałe znaki w linii to wartości wektora - parsuje na liczbę całkowitą i dodaje do X
            X.append([int(v) for v in line[1:len(line)]])

#normalizacja cech
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X = mms.fit_transform(X)

#standaryzacja cech
#from sklearn.preprocessing import StandardScaler
#stdsc = StandardScaler()
#X = stdsc.fit_transform(X)

#konwersja z listy na macierz (dla wygody)
X = np.array(X)
y = np.array(y)

## 2. Przygotowanie danych
Dzieli wczytane dane na dwie części: zbiór uczący i zbiór testowy, gdzie zbiór uczący domyślnie stanowi 80% danych wejściowych, a zbiór testowy 20%.

Zwraca tuple (krotkę) zawierającą:
- training_samples - zbiór wektorów cech treningowych o rozmiarze train_size X ilosc_cech
- test_samples - zbiór wektorów cech testowych o rozmiarze test_size X ilosc_cech
- training_classes - zbiór nazw klas (litery) dla zbioru treningowego
- test_classes - zbiór nazw klas (litery) dla zbioru testowego

In [15]:
def prepareDataSet(Samples, Classes, ratio=.8, random=True):

    #wyznaczenie rozmiarów zbiorów (uczącego i testowego)
    train_size = int(ratio * Samples.shape[0])
    test_size = Samples.shape[0] - train_size

    if random:
        #wymieszanie danych - dzięki temu za każdym razem, gdy będzie budowany nowy klasyfikator,
        #inna część będzie brana do uczenia, a inna do testów
        indices = np.random.permutation(Samples.shape[0])
    else:
        indices = np.arange(Samples.shape[0])

    #tablice wymieszanych indeksów (część do uczenia i część do testów)
    training_idx = indices[:train_size]
    test_idx = indices[train_size:]

    #zbiory wektorów cech (część do uczenia i część do testów)
    training_samples, test_samples = Samples[training_idx,:], Samples[test_idx,:]

    #zbiory klas (część do uczenia i część do testów)
    training_classes, test_classes = Classes[training_idx,], Classes[test_idx,]

    return (training_samples, test_samples, training_classes, test_classes)

## 3. Uruchomienie eksperymentu
Tworzy nowy klasyfikator oparty o metodę minimalno-odległościową kNN i zadane parametry. Następnie 'uczy się' rozpoznawać klasy wykorzystując dane treningowe i przeprowadza test na zbiorze testowym.

Zwraca tuple (krotkę) zawierającą:
- accurancy - dokładność, z jaką udało się dopasować klasy
- verification_time - czas potrzebny na sklasyfikowanie danych

Parametry klasyfikatora:
- n_neighbours - liczba najbliższych sąsiadów; dowolna liczba naturalna (w granicah rozsądku); domyślnie 5
- weights - określenia jakie wagi mają być przypisane do odległości od sąsiada; przyjmuje wartość "uniform" (wszystkie odległości mają taką samą wagę) lub "distance" (im większa odległość, tym mniejsza waga); domyślnie "uniform"
- metric - pod uwagę brane są 3 metryki: "manhattan", "euclidean", "minkowski"; domyślnie "minkowski"
- p - parametr potęgi do metryki "minkowski"; domyślnie 2
- algorithm - algorytm używany do obliczania odległości najbliższego sąsiada; dostępne są: "auto", "ball_tree", "kd_tree", "brute"

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.metrics import accuracy_score
import time

def runExperiment(dataSet, n_neighbors, metric, strategy='none', p=3):
    
    training_samples, test_samples, training_classes, test_classes = dataSet

    start_time = time.time()

    #tworzenie kalsyfikatora
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, weights='uniform', metric=metric, p=p, algorithm='kd_tree') 
    
    #narzucenie strategii decyzyjnej
    if strategy.lower() == 'ovo':
        classifier = OneVsOneClassifier(knn)
    elif strategy.lower() == 'ovr':
        classifier = OneVsRestClassifier(knn)
    else:
        classifier = knn
      
    #"uczenie"
    classifier.fit(training_samples, training_classes)

    #przeprowadzenie testu
    pred_classes = classifier.predict(test_samples)

    #obliczenie dokładności
    accurancy = round(100*accuracy_score(test_classes, pred_classes),4)

    end_time = time.time()
    
    #obliczenie czasu
    verification_time = round(end_time - start_time,4)
    
    return (accurancy, verification_time)

##### Test jednostkowy

In [17]:
#przygotowanie danych z pełnym zestawem cech
dataSet_full = prepareDataSet(X,y, random=False)
#print(dataSet_full[0].shape, dataSet_full[1].shape)

In [18]:
#wykonanie eksperymentu na pełnych danych i wypisanie wyniku
result_full = runExperiment(dataSet_full, 3, 'manhattan', 'ovr')

print('Dokładność: {:.2f}%'.format(result_full[0]))
print('Czas weryfikacji: {:.2f}s'.format(result_full[1]))

Dokładność: 97.96%
Czas weryfikacji: 1.25s


## 4. Redukcja wymiaru przestrzeni cech (metoda PCA)
Celem tego zabiegu jest zmniejszenie ilości cech (rozmiaru wektora). Dzięki temu, klasyfikator ma mniej liczenia, a co za tym idzie, eksperyment wykonuje się szybciej. Jednak zmniejsza się dokładność rozpoznawania klas przez klasyfikator.

In [19]:
from sklearn.decomposition import PCA

#funkcja redukująca przestrzeń cech (pełny wymiar = 16)
def reduceWithPCA(reduction, Samples):

    pca = PCA(n_components=reduction)
    pca.fit(Samples)

    #print(pca.explained_variance_ratio_)  
    #print(pca.explained_variance_)  

    reducedX = pca.transform(Samples)
    return reducedX

##### Test jednostkowy

In [20]:
#przygotowanie danych ze zredukowanym zestawem cech
reducedX = reduceWithPCA(14, X)
dataSet_reduced = prepareDataSet(reducedX,y, random=False)

In [21]:
#wykonanie eksperymentu na zredukowanych danych i wypisanie wyniku
result_reduced = runExperiment(dataSet_reduced, 3, 'euclidean', 'ovo')

print('Dokładność: {:.2f}%'.format(result_reduced[0]))
print('Czas weryfikacji: {:.2f}s'.format(result_reduced[1]))

Dokładność: 98.29%
Czas weryfikacji: 3.53s


## 5. Szukanie optymalnych parametrów

In [22]:
#bardzo nieładna funkcja zwracająca unikalną 'kombinację' wszystkich parametrów
def prepareParams(neighbours, metrics):
    
    strategies = ['OvO', 'OvR']
    params = []
    
    for n in neighbours:
        params.append([n, metrics[0], strategies[0]])
        params.append([n, metrics[0], strategies[1]])
        params.append([n, metrics[1], strategies[0]])
        params.append([n, metrics[1], strategies[1]])
        params.append([n, metrics[2], strategies[0]])
        params.append([n, metrics[2], strategies[1]])
    
    return params
    

In [23]:
#funkcja przeprowadza eksperymenty dla każdej kombinacji parametrów
#zapisuje dane wynikowe do pliku, zwraca najlepszy rezultat, a także wskazuje, który zestaw parametrów jest prawdopodobnie najlepszy
def runOverallExperiment(neighbours, metrics, filename):
    
    params = prepareParams(neighbours, metrics)
    
    accurancies, veri_times, optim_values = [],[],[]
    
    #wpisanie do pliku nagłówków kolumn
    with open(filename + '.csv', 'w') as plik:
        plik.write('liczba_sasiadow;metryka;strategia;dokladnosc;czas_weryfikacji;wartosc_optimum\n')
        
    #wykonanie eksperymentów i uzupełnienie danych
    for p in params:
        result = runExperiment(dataSet_full, p[0], p[1], p[2])
        
        #wyliczanie współczynnika optymalności
        #liczba przed 'result[1]' to waga czasu. Można zwiększyć, aby czas miał większy wpływ przy określaniu optimum
        opt = round((result[0] - .15*result[1])/100,8)
        
        accurancies.append(result[0])
        veri_times.append(result[1])
        optim_values.append(opt)
        
        with open(filename + '.csv', 'a') as plik:
            plik.write(str(p[0]) + ';' + p[1] + ';' + p[2] + ';' + str(result[0]).replace('.',',') + ';' + str(result[1]).replace('.',',') + ';' + str(opt).replace('.',',') + '\n')
        
    #szukanie najwyższej wartości optimum
    opid = optim_values.index(max(optim_values))
    founded_optim = params[opid] + [accurancies[opid]] + [veri_times[opid]] + [optim_values[opid]]
    
    print('Znalezione optimum:',
          '\n\tLiczba sąsiadów:',founded_optim[0],
          '\n\tMetryka:',founded_optim[1],
          '\n\tStrategia:',founded_optim[2],
          '\n\tDokładność:',founded_optim[3],'%',
          '\n\tCzas weryfikacji:',founded_optim[4],'s'
          '\n\tWartość optimum:',founded_optim[5],
         )
    
    return founded_optim
    

In [24]:
#lista testowanych liczb sąsiadów (wedle uznania)
neighbours = [3,5,7,9,11]

#lista testowanych metryk (3)
metrics = ['manhattan', 'euclidean', 'minkowski']

#uruchomienie eksperymentu szukającego optymalnych wartości parametrów
#czas wykonania może być długi, zależnie od podanej liczby sąsiadów i użytych metryk (NIE polecam 'minkowski' w strategii OvO)
#jeśli jesteś gotowy przeprowadzić test całościowy, odkomentuj linijkę poniżej i poczekaj na wynik
result_overall = runOverallExperiment(neighbours, metrics, 'dane_do_wykresow')

Znalezione optimum: 
	Liczba sąsiadów: 3 
	Metryka: euclidean 
	Strategia: OvR 
	Dokładność: 98.2861 % 
	Czas weryfikacji: 1.1044 s
	Wartość optimum: 0.9812044


## 6. Szukanie optymalnie zredukowanej przestrzeni cech

In [25]:
#funkcja przeprowadza eksperymenty dla danych z różnym stopniem redukcji przestrzeni cech
#zapisuje dane wynikowe do pliku, zwraca najlepszy rezultat, a także wskazuje, jaki stopień redukcji jest prawdopodobnie najbardziej optymalny
def runPCAExperiment(reducers, parameters, filename):

    accurancies, veri_times = [],[]
    
    #wpisanie do pliku nagłówków kolumn
    with open(filename + '.csv', 'w') as plik:
        plik.write('przestrzen_cech;dokladnosc;czas_weryfikacji\n')
    
    #wykonanie eksperymentów i uzupełnienie danych
    for rc in reducers:
        
        #przygotowanie danych ze zredukowanym zestawem cech
        reduced_X = reduceWithPCA(rc, X)
        reduced_dataSet = prepareDataSet(reduced_X,y, random=False)
        
        result = runExperiment(reduced_dataSet, parameters[0], parameters[1], parameters[2])
        
        accurancies.append(result[0])
        veri_times.append(result[1])
        
        with open(filename + '.csv', 'a') as plik:
            plik.write(str(rc) + ';' + str(result[0]).replace('.',',') + ';' + str(result[1]).replace('.',',') + '\n')
    
    #wyciąganie takich rezultatów, których dokładność jest nie mniejsza niż 1% optymalnego
    candidates = [acc for acc in accurancies if acc >= result_overall[3]-1.]
    
    #szukanie najlepszego rezultatu
    if(len(candidates) == 0):
        gid = accurancies.index(max(accurancies))
    else:
        gid = accurancies.index(min(candidates))
        
    goal = [reducers[gid]] + [accurancies[gid]] + [veri_times[gid]]
    
    print('Optymalnie zredukowana przestrzeń:',
          '\n\tWartość:',goal[0],
          '\n\tDokładność:',goal[1],
          '\n\tCzas weryfikacji:',goal[2],
         )
    return goal
    

In [26]:
#lista przestrzeni do jakich będziemy redukować
reducers = [16,15,14,13,12,11,10]

#uruchomienie experymentu szukającego optymalnie zredukowanej przestrzeni cech
result_final = runPCAExperiment(reducers, result_overall, 'dane_PCA')

print('\nKońcowy wynik:',
      '\n\tZaoszczędzono:',round(result_overall[4] - result_final[2],4),'sekund',
      '\n\tKosztem:',round(result_overall[3] - result_final[1],4),'% dokładności'
     )

Optymalnie zredukowana przestrzeń: 
	Wartość: 11 
	Dokładność: 97.2973 
	Czas weryfikacji: 0.525

Końcowy wynik: 
	Zaoszczędzono: 0.5794 sekund 
	Kosztem: 0.9888 % dokładności
